In [ ]:
import geemap 
import ee 
import pandas as pd

ee.Authenticate()
ee.Initialize(project="cloud-cover-421214")

In [ ]:
df = pd.read_csv("../FRT_coords/tank_inventory.csv")
df = df.dropna(axis=0, how='any')

features = [
    ee.Feature(ee.Geometry.Rectangle(
        coords=[
            float(r["NW Coor (Long, Lat)"][1:-1].split(", ")[0]), # nw long 
            float(r["SE Coor (Long, Lat)"][1:-1].split(", ")[1]), # se lat 
            float(r["SE Coor (Long, Lat)"][1:-1].split(", ")[0]), # se long 
            float(r["NW Coor (Long, Lat)"][1:-1].split(", ")[1]) # nw lat
        ]), 
        {"state":r["State"], "county":r["County"], "id": r["ID"]}
    )
    # for index, r in df.head(10).iterrows()
    for index, r in df.iterrows()
]

fc = ee.FeatureCollection(features)

In [ ]:

m = geemap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_ee_layer(fc.style(color='red'))
m

In [ ]:
cushing_ok_site = [[-96.76487361449992,35.919767693818464],
[-96.7147484924296,35.919767693818464],
[-96.7147484924296,35.954792502408324],
[-96.76487361449992,35.954792502408324],
[-96.76487361449992,35.919767693818464]]

cushing = ee.Geometry.Polygon(cushing_ok_site)

cloudscores = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED").filterBounds(fc).filterDate("2020-01-05", "2021-01-03").filterBounds(cushing)

m.add_ee_layer(cloudscores.first().clipToCollection(fc))

m